In [ ]:
import json
import gzip
import pandas as pd
import datetime
!pip install transformers
from transformers import pipeline
from google.colab import drive
drive.mount('/content/drive')
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load Data

In [ ]:
# load user reviews data for beauty category
file = '/content/drive/MyDrive/CS 229 Group Project/Data/All_Beauty.jsonl.gz'

data_list = []

with gzip.open(file, 'rt', encoding='utf-8') as fp:
    for line in fp:
        data = json.loads(line.strip())
        data_list.append(data)

Beauty_User_df = pd.DataFrame(data_list)

In [ ]:
Beauty_User_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701528 entries, 0 to 701527
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   rating             701528 non-null  float64
 1   title              701528 non-null  object 
 2   text               701528 non-null  object 
 3   images             701528 non-null  object 
 4   asin               701528 non-null  object 
 5   parent_asin        701528 non-null  object 
 6   user_id            701528 non-null  object 
 7   timestamp          701528 non-null  int64  
 8   helpful_vote       701528 non-null  int64  
 9   verified_purchase  701528 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 48.8+ MB


In [ ]:
# load user reviews data for fashion category
file = '/content/drive/MyDrive/CS 229 Group Project/Data/Amazon_Fashion.jsonl.gz'

data_list = []

with gzip.open(file, 'rt', encoding='utf-8') as fp:
    for line in fp:
        data = json.loads(line.strip())
        data_list.append(data)

Fashion_User_df = pd.DataFrame(data_list)

In [ ]:
Fashion_User_df.info()

In [ ]:
# check user overlap between fashion and beauty categories
print(len(set(Beauty_User_df['user_id']).intersection(set(Fashion_User_df['user_id']))))

In [ ]:
# load meta data for these 2 cateogories
file = '/content/drive/MyDrive/CS 229 Group Project/Data/meta_All_Beauty.jsonl.gz'

data_list = []

with gzip.open(file, 'rt', encoding='utf-8') as fp:
    for line in fp:
        data = json.loads(line.strip())
        data_list.append(data)

Meta_Beauty_df = pd.DataFrame(data_list)

In [ ]:
Meta_Beauty_df.info()

In [ ]:
# load meta data for these 2 cateogories
file = '/content/drive/MyDrive/CS 229 Group Project/Data/meta_Amazon_Fashion.jsonl.gz'

data_list = []

with gzip.open(file, 'rt', encoding='utf-8') as fp:
    for line in fp:
        data = json.loads(line.strip())
        data_list.append(data)

Meta_Fashion_df = pd.DataFrame(data_list)

In [ ]:
Meta_Fashion_df.info()

User Review Data Analysis

In [ ]:
beauty_users = set(Beauty_User_df['user_id'])
fashion_users = set(Fashion_User_df['user_id'])

overlapping_users = beauty_users.intersection(fashion_users)

Overlapping_User_Fashion_df = Fashion_User_df[Fashion_User_df['user_id'].isin(overlapping_users)]

In [ ]:
print(len(beauty_users))

In [ ]:
Overlapping_User_Fashion_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210011 entries, 1 to 2500715
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   rating             210011 non-null  float64
 1   title              210011 non-null  object 
 2   text               210011 non-null  object 
 3   images             210011 non-null  object 
 4   asin               210011 non-null  object 
 5   parent_asin        210011 non-null  object 
 6   user_id            210011 non-null  object 
 7   timestamp          210011 non-null  int64  
 8   helpful_vote       210011 non-null  int64  
 9   verified_purchase  210011 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 16.2+ MB


In [ ]:
print(len(set(Overlapping_User_Fashion_df['user_id'])))

116428


# Slice User Review Data (focus on users with > 100 review only)

In [ ]:
user_review_counts = Overlapping_User_Fashion_df.groupby('user_id').size()

users_with_20_reviews = user_review_counts[user_review_counts > 20].index

users_with_20_reviews_df = Overlapping_User_Fashion_df[Overlapping_User_Fashion_df['user_id'].isin(users_with_20_reviews)]

In [ ]:
users_with_20_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12433 entries, 48 to 2171422
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             12433 non-null  float64
 1   title              12433 non-null  object 
 2   text               12433 non-null  object 
 3   images             12433 non-null  object 
 4   asin               12433 non-null  object 
 5   parent_asin        12433 non-null  object 
 6   user_id            12433 non-null  object 
 7   timestamp          12433 non-null  int64  
 8   helpful_vote       12433 non-null  int64  
 9   verified_purchase  12433 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 983.5+ KB


## save P5Y_Overlapping_User_Fashion_df

In [ ]:
# save data
users_with_20_reviews_df.to_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/users_with_20_reviews_df.csv', index=False)

Meta Product Data Analysis

In [ ]:
# check product overlap between two categories
print(len(set(Meta_Beauty_df['parent_asin']).intersection(set(Meta_Fashion_df['parent_asin']))))

0


In [ ]:
beauty_over_100_reviews_df = Meta_Beauty_df[Meta_Beauty_df['rating_number'] > 100]

beauty_sorted_df = beauty_over_100_reviews_df.sort_values(by='rating_number', ascending=False)

top_100_beauty_df = beauty_sorted_df.head(100)

In [ ]:
top_100_beauty_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 50834 to 97867
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   main_category    100 non-null    object 
 1   title            100 non-null    object 
 2   average_rating   100 non-null    float64
 3   rating_number    100 non-null    int64  
 4   features         100 non-null    object 
 5   description      100 non-null    object 
 6   price            52 non-null     float64
 7   images           100 non-null    object 
 8   videos           100 non-null    object 
 9   store            99 non-null     object 
 10  categories       100 non-null    object 
 11  details          100 non-null    object 
 12  parent_asin      100 non-null    object 
 13  bought_together  0 non-null      object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.7+ KB


In [ ]:
top_100_beauty_df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
50834,All Beauty,"Elle 18 Eye Drama Kajal, Bold Black, 0.35 g",4.2,30206,[],"[Elle 18 Eye Drama Kajal, Bold Black, 0.35 g]",5.4,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Elle 18 Eye Drama Kajal - Review a...,Elle 18,[],"{'Color': 'Black', 'Brand': 'Elle 18', 'Item F...",B01LNT83KK,None
1403,All Beauty,"Maybelline New York Colossal Bold Eyeliner, Bl...",4.3,27308,[],[],8.0,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Elle 18 Eye Drama Kajal - Review a...,Maybelline New York,[],"{'Brand': 'Maybelline New York', 'Finish Type'...",B07S141T2R,None
51573,All Beauty,JIMIRE Fake Eyelashes Fluffy Volume 3D Faux Mi...,4.4,16062,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],JIMIRE,[],"{'Brand': 'JIMIRE', 'Material': 'Faux Mink', '...",B08CVJJ518,None
55874,All Beauty,"THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1...",4.5,15264,"[This product has NO safety seal, Visibly regu...",[High-strength vitamin and mineral blemish for...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Customer Review for the Ordinary N...,THE ORDINARY,[],"{'Brand': 'THE ORDINARY', 'Primary Supplement ...",B01MDTVZTZ,None
32240,All Beauty,"THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1...",4.5,15185,"[This product has NO safety seal, Visibly regu...",[High-strength vitamin and mineral blemish for...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'This transformed my skin! - Great ...,THE ORDINARY,[],"{'Brand': 'THE ORDINARY', 'Primary Supplement ...",B06VSX2B1S,None


In [ ]:
# save data
top_100_beauty_df.to_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/top_100_beauty_df.csv', index=False)

NLP Processing for consumer review data- Sentiment Score

In [ ]:
users_with_20_reviews_df = pd.read_csv("/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/users_with_20_reviews_df.csv")

In [ ]:
users_with_20_reviews_df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,2.0,elastic adjustable bands for charge 4,"Well. I really like the idea, however, they a...",[],B098NYMTYH,B098NYMTYH,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1661808780730,0,True
1,2.0,open shoulder t shirt,"I am 5'7"".. usually wear a 14/16 or 0x/1x.. 38...",[],B095CHR38Z,B095CHR38Z,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1645062075013,2,True
2,5.0,fancy pumpkin headband,Purchased for my sister to use on Halloween......,[],B071HMN7K8,B071HMN7K8,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1635309000822,0,True
3,5.0,fitbit charge 4 bands,I just had to replace my old Fitbit 3 to the c...,[],B08JSNBF49,B08JTPQNYQ,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1618006763715,3,True
4,1.0,Nike Jacket?,The photo doesn't show this but you need a col...,[],B074XF4KG4,B074XF4KG4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1611092662402,0,True


In [ ]:
from textblob import TextBlob

# Example of adding sentiment analysis to review data
def add_sentiment_scores(df):
    df['text'] = df['text'].astype(str)
    df['title'] = df['title'].astype(str)
    # TextBlob sentiment analysis
    df['sentiment_text'] = df['text'].apply(lambda text: TextBlob(text).sentiment.polarity)
    df['sentiment_title'] = df['title'].apply(lambda text: TextBlob(text).sentiment.polarity)
    return df

df_user = add_sentiment_scores(users_with_20_reviews_df)

In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12433 entries, 0 to 12432
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             12433 non-null  float64
 1   title              12433 non-null  object 
 2   text               12433 non-null  object 
 3   images             12433 non-null  object 
 4   asin               12433 non-null  object 
 5   parent_asin        12433 non-null  object 
 6   user_id            12433 non-null  object 
 7   timestamp          12433 non-null  int64  
 8   helpful_vote       12433 non-null  int64  
 9   verified_purchase  12433 non-null  bool   
 10  sentiment_text     12433 non-null  float64
 11  sentiment_title    12433 non-null  float64
dtypes: bool(1), float64(3), int64(2), object(6)
memory usage: 1.1+ MB


In [ ]:
# save data
df_user.to_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/df_user.csv', index=False)

Feature Engineering - avg rating per user

In [ ]:
df_user = pd.read_csv("/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/df_user.csv")

In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12433 entries, 0 to 12432
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             12433 non-null  float64
 1   title              12433 non-null  object 
 2   text               12433 non-null  object 
 3   images             12433 non-null  object 
 4   asin               12433 non-null  object 
 5   parent_asin        12433 non-null  object 
 6   user_id            12433 non-null  object 
 7   timestamp          12433 non-null  int64  
 8   helpful_vote       12433 non-null  int64  
 9   verified_purchase  12433 non-null  bool   
 10  sentiment_text     12433 non-null  float64
 11  sentiment_title    12433 non-null  float64
dtypes: bool(1), float64(3), int64(2), object(6)
memory usage: 1.1+ MB


In [ ]:
average_ratings = df_user.groupby('user_id')['rating'].mean().reset_index()
average_ratings.rename(columns={'rating': 'user_average_rating'}, inplace=True)
df_user = pd.merge(df_user, average_ratings, on='user_id', how='left')

Feature Engineering - Standard Deviation of Ratings

Measure the variability in a user's ratings. A high standard deviation indicates inconsistent opinions, which could be a useful feature.

In [ ]:
std_ratings = df_user.groupby('user_id')['rating'].std().reset_index()

std_ratings.rename(columns={'rating': 'user_std_rating'}, inplace=True)
std_ratings['user_std_rating'].fillna(0, inplace=True)

df_user = pd.merge(df_user, std_ratings, on='user_id', how='left')

<ipython-input-64-027b0a364f83>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  std_ratings['user_std_rating'].fillna(0, inplace=True)


Feature Engineering - Avg sentiment scores per user

In [ ]:
df_user["overall_sentiment"] = (df_user["sentiment_title"] + df_user["sentiment_text"])/2
average_sentiment = df_user.groupby('user_id')['overall_sentiment'].mean().reset_index()
average_sentiment.rename(columns={'overall_sentiment': 'avg_user_sentiment'}, inplace=True)

df_user = pd.merge(df_user, average_sentiment, on='user_id', how='left')

In [ ]:
df_user.describe()

,rating,timestamp,helpful_vote,sentiment_text,sentiment_title,user_average_rating,user_std_rating,overall_sentiment,avg_user_sentiment
count,12433.000000,1.243300e+04,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000
mean,4.170916,1.580842e+12,0.881123,0.251611,0.306703,4.170916,0.919999,0.279157,0.279157
std,1.071818,5.134911e+10,4.897570,0.187715,0.349152,0.463230,0.338188,0.223268,0.091210
min,1.000000,1.168460e+12,0.000000,-1.000000,-1.000000,1.727273,0.000000,-0.660000,0.019048
25%,4.000000,1.560912e+12,0.000000,0.134303,0.000000,3.920000,0.709105,0.116667,0.220690
50%,5.000000,1.583498e+12,0.000000,0.240357,0.312500,4.166667,0.919866,0.285565,0.275737
75%,5.000000,1.616971e+12,0.000000,0.358333,0.550000,4.518519,1.148243,0.434643,0.342838
max,5.000000,1.684155e+12,195.000000,1.000000,1.000000,5.000000,1.942862,1.000000,0.573551


Feature Engineering - Review Count

Count the number of reviews posted by each user as a measure of engagement.

In [ ]:
review_counts = df_user.groupby('user_id').size().reset_index(name='user_review_count')

df_user = pd.merge(df_user, review_counts, on='user_id', how='left')

In [ ]:
df_user.describe()

,rating,timestamp,helpful_vote,sentiment_text,sentiment_title,user_average_rating,user_std_rating,overall_sentiment,avg_user_sentiment,user_review_count
count,12433.000000,1.243300e+04,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000
mean,4.170916,1.580842e+12,0.881123,0.251611,0.306703,4.170916,0.919999,0.279157,0.279157,56.335317
std,1.071818,5.134911e+10,4.897570,0.187715,0.349152,0.463230,0.338188,0.223268,0.091210,55.298541
min,1.000000,1.168460e+12,0.000000,-1.000000,-1.000000,1.727273,0.000000,-0.660000,0.019048,21.000000
25%,4.000000,1.560912e+12,0.000000,0.134303,0.000000,3.920000,0.709105,0.116667,0.220690,26.000000
50%,5.000000,1.583498e+12,0.000000,0.240357,0.312500,4.166667,0.919866,0.285565,0.275737,37.000000
75%,5.000000,1.616971e+12,0.000000,0.358333,0.550000,4.518519,1.148243,0.434643,0.342838,60.000000
max,5.000000,1.684155e+12,195.000000,1.000000,1.000000,5.000000,1.942862,1.000000,0.573551,311.000000


Feature Engineering - avg review word count
Average number of words per review, indicating how expressive or detailed the user is

In [ ]:
df_user['word_count'] = df_user['text'].apply(lambda x: len(str(x).split()))

average_word_count = df_user.groupby('user_id')['word_count'].mean().reset_index()
average_word_count.rename(columns={'word_count': 'user_avg_word_count'}, inplace=True)

# Merge the average word counts with the original dataset (optional)
df_user = pd.merge(df_user, average_word_count, on='user_id', how='left')

Feature Engineering - Helpfulness Ratio
Ratio of helpful votes to total reviews, to capture how valuable others find the user's reviews.

In [ ]:
total_helpful_votes = df_user.groupby('user_id')['helpful_vote'].sum().reset_index()
total_reviews = df_user.groupby('user_id').size().reset_index(name='total_reviews')

helpfulness_data = pd.merge(total_helpful_votes, total_reviews, on='user_id')
helpfulness_data['helpfulness_ratio'] = helpfulness_data['helpful_vote'] / helpfulness_data['total_reviews']

df_user = pd.merge(df_user, helpfulness_data[['user_id', 'helpfulness_ratio']], on='user_id', how='left')

Feature Engineering: Verified Purchases Count

Number of verified purchases, which might indicate a higher trust or authenticity in reviews.

In [ ]:
verified_purchases = df_user[df_user['verified_purchase'] == True]

verified_counts = verified_purchases.groupby('user_id').size().reset_index(name='verified_purchase_count')

df_user = pd.merge(df_user, verified_counts, on='user_id', how='left')

# Fill NaN values with 0 where no verified purchases exist
df_user['verified_purchase_count'].fillna(0, inplace=True)

<ipython-input-71-6218f46dbe9e>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_user['verified_purchase_count'].fillna(0, inplace=True)


In [ ]:
df_user.describe()

,rating,timestamp,helpful_vote,sentiment_text,sentiment_title,user_average_rating,user_std_rating,overall_sentiment,avg_user_sentiment,user_review_count,word_count,user_avg_word_count,helpfulness_ratio,verified_purchase_count
count,12433.000000,1.243300e+04,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000,12433.000000
mean,4.170916,1.580842e+12,0.881123,0.251611,0.306703,4.170916,0.919999,0.279157,0.279157,56.335317,80.978444,80.978444,0.881123,6.304271
std,1.071818,5.134911e+10,4.897570,0.187715,0.349152,0.463230,0.338188,0.223268,0.091210,55.298541,68.557114,51.535665,1.770871,10.760937
min,1.000000,1.168460e+12,0.000000,-1.000000,-1.000000,1.727273,0.000000,-0.660000,0.019048,21.000000,0.000000,1.692308,0.000000,0.000000
25%,4.000000,1.560912e+12,0.000000,0.134303,0.000000,3.920000,0.709105,0.116667,0.220690,26.000000,38.000000,49.090909,0.175439,0.000000
50%,5.000000,1.583498e+12,0.000000,0.240357,0.312500,4.166667,0.919866,0.285565,0.275737,37.000000,66.000000,69.857143,0.333333,1.000000
75%,5.000000,1.616971e+12,0.000000,0.358333,0.550000,4.518519,1.148243,0.434643,0.342838,60.000000,104.000000,101.406250,0.809524,6.000000
max,5.000000,1.684155e+12,195.000000,1.000000,1.000000,5.000000,1.942862,1.000000,0.573551,311.000000,1228.000000,341.521739,19.545455,57.000000


# Drop userless features and save user data

In [ ]:
df_user.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12433 entries, 0 to 12432
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rating                   12433 non-null  float64
 1   title                    12433 non-null  object 
 2   text                     12433 non-null  object 
 3   images                   12433 non-null  object 
 4   asin                     12433 non-null  object 
 5   parent_asin              12433 non-null  object 
 6   user_id                  12433 non-null  object 
 7   timestamp                12433 non-null  int64  
 8   helpful_vote             12433 non-null  int64  
 9   verified_purchase        12433 non-null  bool   
 10  sentiment_text           12433 non-null  float64
 11  sentiment_title          12433 non-null  float64
 12  user_average_rating      12433 non-null  float64
 13  user_std_rating          12433 non-null  float64
 14  overall_sentiment     

In [ ]:
df_user.drop(columns=['title', 'text','images','asin','timestamp','helpful_vote', 'verified_purchase', 'sentiment_text', 'sentiment_title', 'overall_sentiment'], inplace=True)

In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12433 entries, 0 to 12432
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rating                   12433 non-null  float64
 1   parent_asin              12433 non-null  object 
 2   user_id                  12433 non-null  object 
 3   user_average_rating      12433 non-null  float64
 4   user_std_rating          12433 non-null  float64
 5   avg_user_sentiment       12433 non-null  float64
 6   user_review_count        12433 non-null  int64  
 7   word_count               12433 non-null  int64  
 8   user_avg_word_count      12433 non-null  float64
 9   helpfulness_ratio        12433 non-null  float64
 10  verified_purchase_count  12433 non-null  float64
dtypes: float64(7), int64(2), object(2)
memory usage: 1.0+ MB


In [ ]:
# check replicated uer
df_user['user_id'].value_counts()

,count
user_id,
AGZZXSMMS4WRHHJRBUJZI4FZDHKQ,311
AG73BVBKUOH22USSFJA5ZWL7AKXA,210
AEYVPPWR4CIKWX4BGYKCBCDL2CZQ,176
AFQQQ5LGNSQUEBGDCYBAZZE5T3DA,145
AG3HKK7MIKRQ7DOTNDOE63X2WJGQ,145
...,...
AEYKTZXAWOPJG5MGGMKBLRJR6Q3A_1,21
AGUYOGXDDUBFRS6KQ7VIVA43VVNQ,21
AHRUV3HZFPZEPRWEMSYMVBAJXFFQ,21


Feature engineering  - difference betweeen user rating and product avg rating

In [ ]:
Meta_Fashion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826108 entries, 0 to 826107
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   main_category    826108 non-null  object 
 1   title            826108 non-null  object 
 2   average_rating   826108 non-null  float64
 3   rating_number    826108 non-null  int64  
 4   features         826108 non-null  object 
 5   description      826108 non-null  object 
 6   price            50249 non-null   float64
 7   images           826108 non-null  object 
 8   videos           826108 non-null  object 
 9   store            799270 non-null  object 
 10  categories       826108 non-null  object 
 11  details          826108 non-null  object 
 12  parent_asin      826108 non-null  object 
 13  bought_together  0 non-null       object 
dtypes: float64(2), int64(1), object(11)
memory usage: 88.2+ MB


In [ ]:
# check difference betweeen user rating from df_user and product avg rating from meta data
merged_df = df_user.merge(Meta_Fashion_df[['parent_asin', 'average_rating']], on='parent_asin', how='left')

In [ ]:
merged_df['rating_diff'] = merged_df['rating'] - merged_df['average_rating']
avg_diff_per_user = merged_df.groupby('user_id')['rating_diff'].mean().reset_index()
avg_diff_per_user.rename(columns={'rating_diff': 'user_avgrating_diff'}, inplace=True)
df_user = pd.merge(df_user, avg_diff_per_user, on='user_id', how='left')

In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12433 entries, 0 to 12432
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rating                   12433 non-null  float64
 1   parent_asin              12433 non-null  object 
 2   user_id                  12433 non-null  object 
 3   user_average_rating      12433 non-null  float64
 4   user_std_rating          12433 non-null  float64
 5   avg_user_sentiment       12433 non-null  float64
 6   user_review_count        12433 non-null  int64  
 7   word_count               12433 non-null  int64  
 8   user_avg_word_count      12433 non-null  float64
 9   helpfulness_ratio        12433 non-null  float64
 10  verified_purchase_count  12433 non-null  float64
 11  user_avgrating_diff      12433 non-null  float64
dtypes: float64(8), int64(2), object(2)
memory usage: 1.1+ MB


In [ ]:
df_user.drop(columns=['rating', 'parent_asin'], inplace=True)

In [ ]:
# only keep unique user info
unique_user_df = df_user.drop_duplicates(subset='user_id')

In [ ]:
unique_user_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 335 entries, 0 to 12231
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  335 non-null    object 
 1   user_average_rating      335 non-null    float64
 2   user_std_rating          335 non-null    float64
 3   avg_user_sentiment       335 non-null    float64
 4   user_review_count        335 non-null    int64  
 5   word_count               335 non-null    int64  
 6   user_avg_word_count      335 non-null    float64
 7   helpfulness_ratio        335 non-null    float64
 8   verified_purchase_count  335 non-null    float64
 9   user_avgrating_diff      335 non-null    float64
dtypes: float64(7), int64(2), object(1)
memory usage: 28.8+ KB


In [ ]:
# save data
unique_user_df.to_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/unique_user_df.csv', index=False)

# Combine user data with product info

In [ ]:
# import data
unique_user_df = pd.read_csv("/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/unique_user_df.csv")

In [ ]:
top_100_beauty_df = pd.read_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/top_100_beauty_df.csv')

In [ ]:
unique_user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  335 non-null    object 
 1   user_average_rating      335 non-null    float64
 2   user_std_rating          335 non-null    float64
 3   avg_user_sentiment       335 non-null    float64
 4   user_review_count        335 non-null    int64  
 5   word_count               335 non-null    int64  
 6   user_avg_word_count      335 non-null    float64
 7   helpfulness_ratio        335 non-null    float64
 8   verified_purchase_count  335 non-null    float64
 9   user_avgrating_diff      335 non-null    float64
dtypes: float64(7), int64(2), object(1)
memory usage: 26.3+ KB


In [ ]:
top_100_beauty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   main_category    100 non-null    object 
 1   title            100 non-null    object 
 2   average_rating   100 non-null    float64
 3   rating_number    100 non-null    int64  
 4   features         100 non-null    object 
 5   description      100 non-null    object 
 6   price            52 non-null     float64
 7   images           100 non-null    object 
 8   videos           100 non-null    object 
 9   store            99 non-null     object 
 10  categories       100 non-null    object 
 11  details          100 non-null    object 
 12  parent_asin      100 non-null    object 
 13  bought_together  0 non-null      float64
dtypes: float64(3), int64(1), object(10)
memory usage: 11.1+ KB


In [ ]:
print(top_100_beauty_df[['average_rating', 'parent_asin']].describe(include='all'))
print(top_100_beauty_df[['average_rating', 'parent_asin']].isnull().sum())

        average_rating parent_asin
count       100.000000         100
unique             NaN         100
top                NaN  B01LNT83KK
freq               NaN           1
mean          4.435000         NaN
std           0.299958         NaN
min           3.600000         NaN
25%           4.200000         NaN
50%           4.400000         NaN
75%           4.700000         NaN
max           4.800000         NaN
average_rating    0
parent_asin       0
dtype: int64


In [ ]:
prod_ratings = top_100_beauty_df.pivot_table(
    values='average_rating',
    columns='parent_asin',
    aggfunc='mean'
)

prod_ratings_df = pd.DataFrame(prod_ratings)

prod_ratings_df = prod_ratings_df.transpose()
prod_ratings_df.reset_index(inplace=True)
prod_ratings_df.columns = ['parent_asin', 'average_rating']

print(prod_ratings_df.info())
print(prod_ratings_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   parent_asin     100 non-null    object 
 1   average_rating  100 non-null    float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB
None
  parent_asin  average_rating
0  B001DZVELU             4.6
1  B001O1M6II             4.2
2  B004PB8IKO             4.5
3  B0050HW9PW             4.4
4  B005BZQHEC             4.8


In [ ]:
prod_ratings_df = prod_ratings_df.set_index('parent_asin').T
prod_ratings_df.columns = ['avg_rating_' + col for col in prod_ratings_df.columns]

prod_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, average_rating to average_rating
Data columns (total 100 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   avg_rating_B001DZVELU  1 non-null      float64
 1   avg_rating_B001O1M6II  1 non-null      float64
 2   avg_rating_B004PB8IKO  1 non-null      float64
 3   avg_rating_B0050HW9PW  1 non-null      float64
 4   avg_rating_B005BZQHEC  1 non-null      float64
 5   avg_rating_B007E9I11K  1 non-null      float64
 6   avg_rating_B008R5LW5W  1 non-null      float64
 7   avg_rating_B009DGO30K  1 non-null      float64
 8   avg_rating_B00B5AKXXI  1 non-null      float64
 9   avg_rating_B00BXGVIII  1 non-null      float64
 10  avg_rating_B00DQBMN6E  1 non-null      float64
 11  avg_rating_B00E5EX8HY  1 non-null      float64
 12  avg_rating_B00KT2JEJM  1 non-null      float64
 13  avg_rating_B00N7MSRSK  1 non-null      float64
 14  avg_rating_B00P7XF3K2  1 non-null      f

In [ ]:
input_df = unique_user_df.join(prod_ratings_df, how='cross')

In [ ]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Columns: 110 entries, user_id to avg_rating_B0C9CWKY9G
dtypes: float64(107), int64(2), object(1)
memory usage: 288.0+ KB


In [ ]:
# save input_data
input_df.to_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/input_df.csv', index=False)

Prepare y labels

In [ ]:
input_df = pd.read_csv("/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/input_df.csv")

In [ ]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Columns: 110 entries, user_id to avg_rating_B0C9CWKY9G
dtypes: float64(107), int64(2), object(1)
memory usage: 288.0+ KB


In [ ]:
top_100_beauty_df = pd.read_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/top_100_beauty_df.csv')

In [ ]:
# extract unique user_id
unique_user_ids = input_df['user_id'].unique()

In [ ]:
# for beauty user review data, filter the reviews to include only the top 1000 products
top_100_asin = top_100_beauty_df['parent_asin'].unique()

In [ ]:
filtered_beauty = Beauty_User_df[Beauty_User_df['parent_asin'].isin(top_100_asin)]

input_df.set_index('user_id', inplace=True)
filtered_beauty.set_index('user_id', inplace=True)

In [ ]:
pivot_purchase = filtered_beauty.pivot_table(index='user_id', columns='parent_asin', values='verified_purchase', fill_value=False)
pivot_rating = filtered_beauty.pivot_table(index='user_id', columns='parent_asin', values='rating')

In [ ]:
pivot_purchase.describe()

parent_asin,B001DZVELU,B001O1M6II,B004PB8IKO,B0050HW9PW,B005BZQHEC,B007E9I11K,B008R5LW5W,B009DGO30K,B00B5AKXXI,B00BXGVIII,...,B09LHS6RGF,B09X9BG4FC,B0B1W8X4DK,B0B5G9SG6Z,B0B7MCJ618,B0B96MFXSJ,B0BM4GX6TT,B0BMWQQR9W,B0BNWSR278,B0C9CWKY9G
count,23734,23734,23734,23734,23734,23734,23734,23734,23734,23734,...,23734,23734,23734,23734,23734,23734,23734,23734,23734,23734
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,1,2,2,2,2,3,2
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,23730,23614,23639,23723,23018,23733,23662,23718,23733,23732,...,22881,22427,23733,23734,23487,23316,22039,23419,23234,22606


In [ ]:
# Join pivoted data back to input_df
input_df = input_df.join(pivot_purchase.add_prefix('y_verified_purchase_'), on='user_id')
input_df = input_df.join(pivot_rating.add_prefix('y_rating_'), on='user_id')

In [ ]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 335 entries, AFZUK3MTBIBEDQOPAK3OATUOUKLA to AESH4UBOA6VWZ36CIYMP4JVYPOLA
Columns: 309 entries, user_average_rating to y_rating_B0C9CWKY9G
dtypes: float64(207), int64(2), object(100)
memory usage: 811.3+ KB


In [ ]:
for asin in top_100_asin:
       # Find the row in Meta_Beauty_df where parent_asin matches asin
       row = Meta_Beauty_df.loc[Meta_Beauty_df['parent_asin'] == asin]
       if not row.empty:
           average_rating = row['average_rating'].iloc[0]
           if f'y_rating_{asin}' in input_df.columns:
               input_df[f'y_rating_{asin}'].fillna(average_rating, inplace=True)
           else:
               input_df[f'y_rating_{asin}'] = average_rating
       else:
           print(f"ASIN {asin} not found in Meta_Beauty_df")

<ipython-input-108-904fad878716>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  input_df[f'y_rating_{asin}'].fillna(average_rating, inplace=True)


In [ ]:
# check data
rating_columns = [col for col in input_df.columns if col.startswith('y_rating_')]

print(input_df[rating_columns].info())

<class 'pandas.core.frame.DataFrame'>
Index: 335 entries, AFZUK3MTBIBEDQOPAK3OATUOUKLA to AESH4UBOA6VWZ36CIYMP4JVYPOLA
Data columns (total 100 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   y_rating_B001DZVELU  335 non-null    float64
 1   y_rating_B001O1M6II  335 non-null    float64
 2   y_rating_B004PB8IKO  335 non-null    float64
 3   y_rating_B0050HW9PW  335 non-null    float64
 4   y_rating_B005BZQHEC  335 non-null    float64
 5   y_rating_B007E9I11K  335 non-null    float64
 6   y_rating_B008R5LW5W  335 non-null    float64
 7   y_rating_B009DGO30K  335 non-null    float64
 8   y_rating_B00B5AKXXI  335 non-null    float64
 9   y_rating_B00BXGVIII  335 non-null    float64
 10  y_rating_B00DQBMN6E  335 non-null    float64
 11  y_rating_B00E5EX8HY  335 non-null    float64
 12  y_rating_B00KT2JEJM  335 non-null    float64
 13  y_rating_B00N7MSRSK  335 non-null    float64
 14  y_rating_B00P7XF3K2  335 non-null    float

In [ ]:
# Filter the columns that start with 'y_verified_purchase'
rating_columns = [col for col in input_df.columns if col.startswith('y_verified_purchase')]

input_df[rating_columns] = input_df[rating_columns].fillna(False)

# Verify the changes by checking again for NaN values
na_check_updated = input_df[rating_columns].isna().any()
print(na_check_updated)

y_verified_purchase_B001DZVELU    False
y_verified_purchase_B001O1M6II    False
y_verified_purchase_B004PB8IKO    False
y_verified_purchase_B0050HW9PW    False
y_verified_purchase_B005BZQHEC    False
                                  ...  
y_verified_purchase_B0B96MFXSJ    False
y_verified_purchase_B0BM4GX6TT    False
y_verified_purchase_B0BMWQQR9W    False
y_verified_purchase_B0BNWSR278    False
y_verified_purchase_B0C9CWKY9G    False
Length: 100, dtype: bool


<ipython-input-110-013ebc13a792>:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_df[rating_columns] = input_df[rating_columns].fillna(False)


In [ ]:
rating_columns = [col for col in input_df.columns if col.startswith('y_verified_purchase')]
verified_purchase_counts = {}
for column in rating_columns:
    verified_purchase_counts[column] = input_df[column].value_counts()

verified_purchase_counts_df = pd.DataFrame(verified_purchase_counts)

# Print the result
print(verified_purchase_counts_df)

       y_verified_purchase_B001DZVELU  y_verified_purchase_B001O1M6II  \
False                           335.0                           335.0   
1.0                               NaN                             NaN   

       y_verified_purchase_B004PB8IKO  y_verified_purchase_B0050HW9PW  \
False                           335.0                           335.0   
1.0                               NaN                             NaN   

       y_verified_purchase_B005BZQHEC  y_verified_purchase_B007E9I11K  \
False                           335.0                           335.0   
1.0                               NaN                             NaN   

       y_verified_purchase_B008R5LW5W  y_verified_purchase_B009DGO30K  \
False                           335.0                           335.0   
1.0                               NaN                             NaN   

       y_verified_purchase_B00B5AKXXI  y_verified_purchase_B00BXGVIII  ...  \
False                           335.0     

In [ ]:
input_df.head()

,user_average_rating,user_std_rating,avg_user_sentiment,user_review_count,word_count,user_avg_word_count,helpfulness_ratio,verified_purchase_count,user_avgrating_diff,avg_rating_B001DZVELU,...,y_rating_B09LHS6RGF,y_rating_B09X9BG4FC,y_rating_B0B1W8X4DK,y_rating_B0B5G9SG6Z,y_rating_B0B7MCJ618,y_rating_B0B96MFXSJ,y_rating_B0BM4GX6TT,y_rating_B0BMWQQR9W,y_rating_B0BNWSR278,y_rating_B0C9CWKY9G
user_id,,,,,,,,,,,,,,,,,,,,,
AFZUK3MTBIBEDQOPAK3OATUOUKLA,4.000000,1.287301,0.087327,36,115,134.388889,1.944444,34.0,0.022222,4.6,...,4.5,4.6,4.4,4.4,4.6,4.6,4.4,4.2,4.4,4.8
AHV6QCNBJNSGLATP56JAWJ3C4G2A,4.128205,0.709105,0.323301,78,100,105.089744,0.653846,4.0,0.084615,4.6,...,4.5,4.6,4.4,4.4,4.6,4.6,4.4,4.2,4.4,4.8
AFJBKPK5W56XWSNPQU2WW66ISWYQ,4.278689,0.777413,0.369100,61,57,47.852459,0.098361,0.0,0.209836,4.6,...,4.5,4.6,4.4,4.4,4.6,4.6,4.4,4.2,4.4,4.8
AHJQPUQLSQZE6LMIUMY7WNRXCQQQ,3.682927,1.293040,0.177344,41,43,121.439024,0.512195,3.0,0.000000,4.6,...,4.5,4.6,4.4,4.4,4.6,4.6,4.4,4.2,4.4,4.8
AFXF3EGQTQDXMRLDWFU7UBFQZB7Q,3.483871,0.987495,0.212081,62,63,141.129032,0.451613,5.0,-0.491935,4.6,...,4.5,4.6,4.4,4.4,4.6,4.6,4.4,4.2,4.4,4.8


In [ ]:
# save data
input_df.to_csv('/content/drive/MyDrive/CS 229 Group Project/Data/filtered data/input_labels_df.csv', index=False)

## Split Data

In [ ]:
input_df = pd.read_csv("/content/drive/MyDrive/CS 229 Group Project/Data/input_labels_df.csv")

<ipython-input-3-a196022f355a>:1: DtypeWarning: Columns (1010,1011,1013,1014,1017,1019,1020,1022,1023,1026,1028,1029,1032,1033,1034,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1055,1056,1057,1058,1060,1063,1064,1067,1069,1070,1071,1072,1074,1075,1076,1077,1079,1080,1081,1082,1085,1088,1089,1091,1092,1093,1094,1096,1097,1099,1101,1102,1103,1104,1105,1106,1108,1110,1111,1113,1115,1116,1117,1118,1121,1122,1123,1124,1125,1127,1128,1129,1131,1132,1136,1139,1140,1141,1142,1143,1145,1146,1147,1148,1150,1151,1152,1154,1159,1160,1163,1164,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1178,1179,1180,1182,1183,1184,1185,1186,1188,1189,1190,1192,1193,1197,1199,1200,1201,1202,1203,1205,1206,1207,1208,1210,1211,1212,1216,1217,1218,1219,1220,1224,1227,1228,1230,1231,1232,1234,1235,1236,1237,1238,1239,1240,1241,1242,1243,1244,1245,1247,1248,1249,1252,1253,1256,1259,1260,1261,1263,1264,1266,1267,1268,1269,1271,1272,1273,1274,1275,1276,1277,1278,1280,1281,1283,1284,128

In [ ]:
pip install scikit-learn


In [ ]:
from sklearn.model_selection import train_test_split

feature_columns = [col for col in input_df.columns if not col.startswith('y_')]
X = input_df[feature_columns]  # Features
y = input_df[[col for col in input_df.columns if col.startswith('y_')]]

In [ ]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80981 entries, 0 to 80980
Columns: 2000 entries, y_verified_purchase_1891129716 to y_rating_B0CDNZ7F2V
dtypes: bool(223), float64(1000), object(777)
memory usage: 1.1+ GB


In [ ]:
pip install iterative-stratification